# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Basic Usage

In [ ]:
import * as setup from "./setup.ts";
import { createAgent } from "langchain";

const agent = await createAgent({
    model: "anthropic:claude-sonnet-4-5-20250929",
    systemPrompt: "You are a full-stack comedian",
});

Now let's invoke the agent with a simple message.


In [ ]:
import { HumanMessage } from "langchain";

const humanMessage = new HumanMessage("Hello, how are you?");
const result = await agent.invoke({ messages: [humanMessage] });

The result contains a `messages` array. Let's see what the agent responded with:


In [ ]:
console.log(result.messages.at(-1).content)

We can iterate through all messages to see the full conversation history:


In [ ]:
for (const message of result.messages) {
    displayMessage(message)
}

### Altenative formats
#### Strings

In [ ]:
const agent = createAgent({
    model: "anthropic:claude-sonnet-4-5-20250929",
    systemPrompt: "You are a terse sports poet.",
})

Instead of using message classes, you can pass a plain string:


In [ ]:
const result = await agent.invoke({
    messages: "Tell me about baseball"
})
console.log(result.messages.at(-1).content)

#### Object

You can also pass an object with `role` and `content`:


In [ ]:
const result = await agent.invoke({
    messages: {
        role: "user",
        content: "Write a haiku about sprinters"
    }
})
console.log(result.messages.at(-1).content)

There are multiple roles you can use in message objects:


There are multiple roles:
```ts
const messages = [
    { role: "system", content: "You are a sports poetry expert who completes haikus that have been started" },
    { role: "user", content: "Write a haiku about sprinters" },
    { role: "assistant", content: "Feet don't fail me..." }
]
```

#### Classes

Finally, you can use the message classes for explicit type control:


In [ ]:
import { HumanMessage } from "langchain";

const result = await agent.invoke({
    messages: [new HumanMessage("Write a haiku about sprinters")]
})
console.log(result.messages.at(-1).content)

## Output Format
### messages

First, let's define a tool that checks if a haiku has the correct number of lines:


In [ ]:
import { z } from "zod";
import { tool } from "langchain";

const checkHaikuLines = tool(({ text }) => {
    const lines = text.split("\n").map(line => line.trim()).filter(Boolean);
    console.log(`checking haiku, it has ${lines.length} lines:\n ${text}`);
    if (lines.length !== 3) {
        return `Incorrect! This haiku has ${lines.length} lines. A haiku must have exactly 3 lines.`;
    }
    return "Correct, This haiku has 3 lines.";
}, {
    name: "check_haiku_lines",
    description: "Checks if the given haiku text has exactly 3 lines.",
    schema: z.object({
        text: z.string().describe("The haiku text to check"),
    }),
});

Now we'll create an agent that uses this tool to validate its haikus:


In [ ]:
import * as setup from "./setup.ts";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "anthropic:claude-sonnet-4-5-20250929",
    tools: [checkHaikuLines],
    systemPrompt: "You are a sports poet who only writes Haiku. You always check your work."
})

Let's ask the agent to write a poem (which it will write as a haiku and check):


In [ ]:
const result = await agent.invoke({
    messages: "Please write me a poem"
})

In [ ]:
result.messages.at(-1).content

The agent wrote a valid haiku! Now let's check the message count:


In [ ]:
console.log(result["messages"].length)

Four messages total! Let's see them all:


In [ ]:
for (const message of result.messages) {
    displayMessage(message)
}

Notice the workflow: human → AI with tool call → tool result → final AI response.


### Other useful information

The full result object shows all messages with their metadata:


In [ ]:
result

Each individual message has rich metadata:


In [ ]:
result.messages.at(-1)

The `usage_metadata` tracks token consumption including reasoning tokens:


In [ ]:
result.messages.at(-1).usage_metadata

Finally, `response_metadata` has model-specific information like finish reason and model name:


In [ ]:
result.messages.at(-1).response_metadata

### Try it on your own!
Change the system prompt, use the `displayMessage` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [ ]:
import { createAgent } from "langchain";

const agent = createAgent({
    model: "anthropic:claude-sonnet-4-5-20250929",
    tools: [checkHaikuLines],
    systemPrompt: "Your SYSTEM prompt here"
})
const result = await agent.invoke({
    messages: "Your HUMAN message here"
})
result.messages.at(-1)